# N2O forcing


From O'Connor et al. 2021:

- +0.25 W m-2

no account how this breaks down into direct forcing and ozone contributions in any of Fiona's papers, but from Thornhill I have 0.04 as the contribution to ozone, so direct forcing is 0.21

In [1]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import pandas as pd
import pooch
from scipy.optimize import root

In [2]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [3]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = [
    'CH4',
    'N2O',
    'CO2',
    'Equivalent effective stratospheric chlorine',
    'VOC',
    'NOx',
    'Ozone',
]
species, properties = read_properties(species=species)
properties['CH4']['input_mode'] = 'concentration'
properties['N2O']['input_mode'] = 'concentration'
properties['CO2']['input_mode'] = 'concentration'
properties['Equivalent effective stratospheric chlorine']['input_mode'] = 'concentration'
properties

{'CH4': {'type': 'ch4',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True},
 'N2O': {'type': 'n2o',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True},
 'CO2': {'type': 'co2',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': False},
 'Equivalent effective stratospheric chlorine': {'type': 'eesc',
  'input_mode': 'concentration',
  'greenhouse_gas': False,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True},
 'VOC': {'type': 'other slcf',
  'input_mode': 'emissions',
  'greenhouse_gas': False,
  'aerosol_chemistry_from_emissions': True,
  'aerosol_chemistry_from_concentration': False},
 'NOx': {'type': 'other slcf',
  'input_mode': 'emissions',
  'green

In [4]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [5]:
rcmip_concentration_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-concentrations-annual-means-v5-1-0.csv",
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)
conc_df = pd.read_csv(rcmip_concentration_file)

In [6]:
voc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|VOC')&(emis_df['Region']=='World'),'1850'].values[0]
nox = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|NOx')&(emis_df['Region']=='World'),'1850'].values[0]

co2 = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|CO2')&(conc_df['Region']=='World'),'1850'].values[0]
n2o = conc_df.loc[(conc_df['Scenario']=='ssp245')&(conc_df['Variable']=='Atmospheric Concentrations|N2O')&(conc_df['Region']=='World'),'1850':'2015'].values.squeeze()
n2o[-2] = n2o[-1]
ch4 = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|CH4')&(conc_df['Region']=='World'),'1850'].values[0]

In [7]:
target_forcing = 0.21

def erf_rootfinder(x):
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    f.fill_species_configs()
    
    # 1850 baselines
    f.species_configs['baseline_emissions'].loc[dict(specie='VOC')] = voc
    f.species_configs['baseline_emissions'].loc[dict(specie='NOx')] = nox
    f.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = ch4
    f.species_configs['baseline_concentration'].loc[dict(specie='N2O')] = n2o[0]
    f.species_configs['baseline_concentration'].loc[dict(specie='CO2')] = co2
    f.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0
    
#    f.fill_species_configs()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    
    # fill emissions and concentrations
    f.emissions.loc[dict(specie='VOC')] = voc
    f.emissions.loc[dict(specie='NOx')] = nox
    f.concentration.loc[dict(specie='CH4')] = ch4
    f.concentration.loc[dict(specie='N2O')] = n2o[:, None, None]
    f.concentration.loc[dict(specie='CO2')] = co2
    f.concentration.loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0
    f.temperature[:] = 0

    # pre-calibrated for methane and ozone runs: do not adjust
    f.species_configs['forcing_scale'].loc[dict(specie="CH4")] = 1.11547955
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="CH4")] = 1.27049657e-04
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="Equivalent effective stratospheric chlorine")] = -0.00029119797470220245
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="N2O")] = 0.0007481397748679878
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="VOC")] = 0.0006596999582126578
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="NOx")] = 0.000984642055228785
    f.species_configs['forcing_temperature_feedback'].loc[dict(specie="Ozone")] = -0.079  # Thornhill et al 2021 feedbacks

    # this is the variable we are changing: N2O scale factor
    f.species_configs['forcing_scale'].loc[dict(specie="N2O")] = x
    
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0) 
    f.run(progress=False)
    return f.forcing[-1, 0, 0, 1] - target_forcing

In [8]:
sol = root(erf_rootfinder, 1.09)

In [11]:
sol.x[0]

1.0845323023219764